In [1]:
import numpy as np
import pandas as pd
import transformers
import json
import warnings
from datasets import Dataset
warnings.filterwarnings("ignore")

In [2]:
with open('/kaggle/input/stanford-question-answering-dataset/train-v1.1.json') as train_file:
    train_data = json.load(train_file)

with open('/kaggle/input/stanford-question-answering-dataset/dev-v1.1.json') as dev_file:
    dev_data = json.load(dev_file)

In [3]:
def prepare_dataset(data):
    contexts = []
    questions = []
    answers = []

    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]  # Take the first answer
                answer['text'] = answer['text']
                answer['answer_start'] = answer['answer_start']

                contexts.append(context)
                questions.append(question)
                answers.append(answer)
    
    return Dataset.from_dict({'context': contexts, 'question': questions, 'answers': answers})

In [4]:
train_dataset = prepare_dataset(train_data)
dev_dataset = prepare_dataset(dev_data)

In [5]:
from transformers import AutoTokenizer

model_name='distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples['question']]
    inputs = tokenizer(
        questions,
        examples['context'],
        max_length=384,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True,  # Now supported by the fast tokenizer
        return_tensors="pt"
    )
    
    start_positions = []
    end_positions = []
    
    for i, answer in enumerate(examples['answers']):
        start_positions.append(answer['answer_start'])
        end_positions.append(answer['answer_start'] + len(answer['text']))
    
    inputs.update({
        "start_positions": start_positions,
        "end_positions": end_positions,
    })
    
    return inputs

In [7]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_dev_dataset = dev_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,5.194200,5.069376
2,4.843800,4.813616


TrainOutput(global_step=10950, training_loss=5.1704162207477165, metrics={'train_runtime': 3848.8393, 'train_samples_per_second': 45.52, 'train_steps_per_second': 2.845, 'total_flos': 1.7167621364554752e+16, 'train_loss': 5.1704162207477165, 'epoch': 2.0})

In [13]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer,device=0)

In [49]:
context="I am Aditya Das. I study at NMIMS"
question='Where does Aditya study ?'

result = qa_pipeline({
    'context': context,
    'question': question
})


In [50]:
print("Prediction:", result)

Prediction: {'score': 0.009663555771112442, 'start': 28, 'end': 33, 'answer': 'NMIMS'}


In [51]:
predicted_start = result['start']
predicted_end = result['end']


true_answer = "NMIMS"
true_start = context.find(true_answer)
true_end = true_start + len(true_answer)


In [52]:
def compute_iou(pred, ref):
    pred_tokens = set(range(pred['start_positions'], pred['end_positions']))
    ref_tokens = set(range(ref['start_positions'], ref['end_positions']))
    intersection = len(pred_tokens & ref_tokens)
    union = len(pred_tokens | ref_tokens)
    return intersection / union if union != 0 else 0

In [53]:
pred = {'start_positions': predicted_start, 'end_positions': predicted_end}
ref = {'start_positions': true_start, 'end_positions': true_end}

# Compute token-level IoU
iou_score = compute_iou(pred, ref)

In [54]:
print("Prediction:", result['answer'])
print("True Answer:", true_answer)
print("Token-level IoU:", iou_score)

Prediction: NMIMS
True Answer: NMIMS
Token-level IoU: 1.0
